## Import Python Module to Create/Upload New Data Instances
### (Persona: Data Engineer)

Use metaflow-based data model

In [ ]:
import pandas as pd
import opal.flow
import metaflow

ch10_name = "652200101100441"   

## Parse and Translate NASA Chapter 10 MilStd1553 and ARINC429 Data
### (Persona: Data Engineer)

In [ ]:
!python tip_parse_flow.py run --c10 /opt/data/telemetry-demo-data/{ch10_name}.ch10

parse_flow = metaflow.Flow("TipParseFlow")
parse_id = parse_flow.latest_successful_run.id

!python tip_translate_flow.py run --parsed {parse_id} --type MILSTD1553 --dts DTS1553_Synth_Nasa.yaml
!python tip_translate_flow.py run --parsed {parse_id} --type ARINC429 --dts DTS429_Synth_NASA.yaml

## Segue To OPAL Catalog

1. (Upper left) Select "File" --> "Hub Control Panel"
2. (Hub Control Panel Tab) Select "Services" --> "opal-catalog"
3. (OPAL Catalog Tab) Select "Login with Keycloak"
4. Click "Edit Columns" button and add "CH10 NAME" and "VALIDATED DATETIME"
5. Place cursor in "Search Catalog" field
6. Type "ch10", arrow down to select suggestion "kind_metadata.ch10_name", then press Enter
7. Search Catalog for ch10 name "652200101100441" -- No entry! Data have not been published.
8. Clear search bar and Enter to reset search
9. Return to notebook tab, see "Publish Datasets To Searchable Catalog"

## Publish Datasets to Searchable Catalog
### (Persona: Data Engineer)

In [ ]:
print(f"Publishing parsed run {parse_flow.latest_successful_run.id}")
print(opal.flow.publish_run(parse_flow.latest_successful_run))

translate_flow = metaflow.Flow("TipTranslateFlow")
for run in translate_flow:
    if run.successful and run.data.parse_run.id == parse_id:
        print(f"Publishing translated run {run.id}")
        print(opal.flow.publish_run(run))

## View Published Data in Catalog

1. Return to OPAL Catalog tab
2. Place cursor in "Search Catalog" field
3. Type "ch10", arrow down to select suggestion "kind_metadata.ch10_name", then press Enter
4. Cursor is placed in quotes, type "441" to search the relevant portion of "652200101100441", press Enter
5. Notice the relevant ch10 by the "VALIDATED DATETIME" column
6. Clear the search text and Enter
7. Type "type" then down arrow to select "kind_type" parameter, followed by Enter
8. Search for "tip_translated"
9. Narrow search: 
    - place cursor after "tip_translate"
    - hit space bar then select "AND"
    - type "validated" and select "validated_datetime"
    - type today's date "2022-05-17" and Enter
10. Click on links in "Kind ID" column to view metadata

## Read Datasets from Catalog
### (Persona: Analyst)
#### Latitude/Longitude/Altitude (1553)

Run metadata automatically included in the catalog can be searched

In [ ]:
import opal.query
inst = opal.query.search.Instance()

arinc429_result = inst._with("parse_pointer", parse_id)._and("translate_type", "ARINC429").search().all()[0]
arinc429_id = arinc429_result["kind_id"]
mil1553_id = inst._with("parse_pointer", parse_id)._and("translate_type", "MILSTD1553").search().all()[0]["kind_id"]

display(arinc429_result)

# Analyze Data in New Dataset
Latitude/Longitude/Altitude (1553)

In [ ]:
import matplotlib.pyplot as plt
import os

# Construct the path to the parquet file in S3
s3 = opal.flow.minio_s3fs()
parquet_path = translate_flow[mil1553_id].data.data_files['translated_data']
parquet_path = os.path.join(parquet_path, "parse_data_translated", "NAV.parquet")

# Read it with Pandas
df_1553 = pd.read_parquet(parquet_path, filesystem=s3)

# Filter out invalid lat/long/altitude measurements
where_valid = df_1553[df_1553["NAV-0110"] & df_1553["NAV-0111"]]
plt.rcParams.update({"font.size":18})

# plot
fig, ax = plt.subplots()
ax = where_valid.plot(
    kind="scatter", title="Aircraft Position (1553)",
    x="NAV-23", y="NAV-21", c="NAV-25", s=1,
    cmap="viridis", figsize=(15, 10), ax=ax)

ax.set_aspect("equal")
ax.set_xlabel("Longitude [deg]")
ax.set_ylabel("Latitude [deg]")
plt.gcf().get_axes()[1].set_ylabel("Altitude [ft]")
None

#### Engine Status (ARINC429) with Altitude (1553)

In [ ]:
arinc429_path = translate_flow[arinc429_id].data.data_files['translated_data']
arinc429_path = os.path.join(arinc429_path, "parse_data_translated")

turbine_speed_path = os.path.join(arinc429_path, "Engine_Turbine_RPM_40.parquet")
exhaust_temp_path = os.path.join(arinc429_path, "Exhaust_Gas_Temperature_40.parquet")

df_turbine_speed = pd.read_parquet(turbine_speed_path, filesystem=s3)
df_exhaust_temp = pd.read_parquet(exhaust_temp_path, filesystem=s3)

ax = df_turbine_speed.plot(kind="scatter", x="time", y="N2_RPM", figsize=(13, 10), c='k', label="Turbine Speed [RPM, percent max.]")
ax.set_ylabel(r'Turbine Speed AND Exhaust Temp', c='k')
ax.tick_params(axis='y', colors='k')
df_exhaust_temp.plot(kind="line", x="time", y="Exhaust_Gas_Temperature", ax=ax, c='k', label="Exhaust Temp [deg C]")
ax2 = where_valid.plot(kind="line", x="time", y="NAV-25", secondary_y=True, c='b', ax=ax)
ax2.set_ylabel("Altitude [ft]", c='b')
ax2.yaxis.label.set_color('blue')
ax2.tick_params(axis='y', colors='blue')
ax.set_xlabel("Time [Epoch, ns]")
ax.legend()
title = ax.set_title("Engine Data (ARINC429) as Function of Altitude (1553)")
None

# Delete data for next run

In [ ]:
opal.flow.delete_run_data(parse_flow[parse_id])
opal.flow.delete_run_data(translate_flow[arinc429_id])
opal.flow.delete_run_data(translate_flow[mil1553_id])